In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#!pip install fastai2

In [ ]:
#conda install pytorch torchvision cudatoolkit=10.2 -c pytorch

In [ ]:
from fastai2.vision.all import *
from fastai2.vision import *
from fastai2.metrics import *
from fastai2.basics import *
from fastai2.torch_basics import *
from fastai2.callback.all import *
from fastai2.test_utils import *
from sklearn.metrics import roc_auc_score
import pandas as pd


In [ ]:
#jpeg folder, train.csv and test.csv should in the path which will be mentioned below
path = "/home/gl-403/Machine_learning/useful_code/malignant/"

In [ ]:
train_info = pd.read_csv(path + 'train.csv')

In [ ]:
train_info.head()

In [ ]:
train_info.isna().any()

In [ ]:
#drop all rows having NaN value
train_info = train_info.dropna()
train_info.isna().any()


In [ ]:
train_info = train_info[:100]
len(train_info)

In [ ]:
tfm=aug_transforms(do_flip=True, flip_vert=True, max_rotate=45.0, max_zoom=1.1, size=128, max_lighting=0.2, max_warp=0.4, p_affine=0.75, p_lighting=0.75, xtra_tfms=None, mode='bilinear')

In [ ]:
#item_tfms uses CPU and # batch_tfms uses GPU
dls = ImageDataLoaders.from_df(train_info, label_col='target',path=path, folder='jpeg/train', suff='.jpg', seed=42, item_tfms=Resize(128, method=ResizeMethod.Pad), batch_tfms = [*tfm, Normalize.from_stats(*imagenet_stats)],bs=10,num_workers=0)

In [ ]:
len(dls.train_ds)

In [ ]:
dls.show_batch(max_n=12, nrows=2, ncols=6)

In [ ]:
roc_auc = RocAuc()
metrics = [accuracy,roc_auc]

In [ ]:
learn = cnn_learner(dls, models.resnet34, metrics=metrics)


In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot_lr_find()

In [ ]:
lr = 0.01

In [ ]:
learn.fit_one_cycle(1, lr)

In [ ]:
learn.save('stage-1-rn34')

In [ ]:
learn.export()

In [ ]:
learn.load('stage-1-rn34')

In [ ]:
learn.unfreeze()

In [ ]:
#learn.lr_find()

In [ ]:
#learn.recorder.plot()

In [ ]:
lr = 0.02

In [ ]:
learn.fit_one_cycle(1, slice(1e-5, lr/5))

In [ ]:
learn.save('stage-2-rn34')

In [ ]:
learn.export()

In [ ]:
learn.show_results()

In [ ]:
interp = Interpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(12)

In [ ]:
test_info = pd.read_csv(path + "test.csv")

In [ ]:
test_dl = dls.test_dl(test_info)

In [ ]:
learn = load_learner(path)


In [ ]:
preds, y = learn.get_preds(dl=test_dl)

In [ ]:
acc = accuracy(preds, y)

In [ ]:
acc

In [ ]:
#thresh = 0.2
#labelled_preds = []
#for pred in preds:
#    for i,p in enumerate(pred):
#        if p > thresh:
#            labelled_preds.append(learn.data.classes[i])
#labelled_preds

In [ ]:
preds

In [ ]:
thresh = 0.5
labelled_preds = []
for pred in preds:
    for i,p in enumerate(pred):
        if p > thresh:
            labelled_preds.append(learn.data.classes[i])

In [ ]:
labelled_preds

In [ ]:
#interp = ClassificationInterpretation.from_learner(learn, ds_type=DatasetType.Train)

In [ ]:
#interp.plot_confusion_matrix()

In [ ]:
fnames = [f[:-4] for f in learn.data.test_ds.items]

In [ ]:
df = pd.DataFrame({'image_name':fnames, 'tags':labelled_preds}, columns=['image_name', 'tags'])


In [ ]:
df.to_csv(path + 'submission.csv', index=False)
